In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 

from PIL import Image

import torch
import torch.nn.functional as F 
from torchvision import models, transforms

import urllib.request

**Load Model and Sample Image**

In [ ]:
def process_image(img_path):
    
    """Load and process image"""
    
    img = Image.open(img_path).convert("RGB")

    #Transforms used by imagenet models
    transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.465, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    return transform(img).unsqueeze(0)

def display_output(output, n=5):

    """Display the top categories predicted by the model."""

    #Catagories Download
    url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
    urllib.request.urlretrieve(url, "imagenet_classes.txt")

    with open("imagenet_classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]

    #Show top categories per image 
    prob = torch.nn.functional.softmax(output[0], dim = 0)
    top_prob, top_cat = torch.topk(prob, n)

    for i in range (top_prob.size(0)):
        print(categories[top_cat[i]], top_prob[i].item())

    return top_cat[0]

In [ ]:
#Load image
img_path = "/home/irfan/Guided-Backpropagation/n01491361_tiger_shark.JPEG"
img = Image.open(img_path).convert("RGB")

plt.imshow(img)
plt.axis("off")

In [ ]:
#Load pre-traind model
model = models.vgg16(pretrained=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.eval();

In [ ]:
#Preprocess the image
orig_img_tn = process_image(img_path)
orig_img_tn = orig_img_tn.to(device)

#Clone the tensor and enable gradient tracking
img_tensor = orig_img_tn.clone()
img_tensor.requires_grad_() 

pred = model(img_tensor)

display_output(pred, n=5)

In [ ]:
#Reset gradient
model.zero_grad()

#select class with the highest probability
target_class = pred.argmax()

#compute gradient w.r.t. to logit y performing backward pass
pred[:, target_class].backward()

In [ ]:
#Get the gradients
standard_backprop_grads = img_tensor.grad.detach().cpu().numpy()
print(standard_backprop_grads.shape)

In [ ]:
def process_grad(grads_in, activation="None", skew=True, normalize=True, greyscale=False):

    """Process the gradeint for visualization"""

    #copy the gradients
    grads = np.copy(grads_in)

    #Transpose the gradients
    if len(grads.shape) >= 3:
        grads = np.transpose(grads, (1,2,0))

    #Get the absolute value of the gradients
    if activation == "relu":
        grads = np.maximum(0, grads)
    elif activation == "abs":
        grads = np.abs(grads)
    else:
        grads = grads

    #normalize the gradients
    if normalize:
        grads -= np.min(grads)
        grads /= (np.max(grads)+1.e-9)

    #skew the gradients
    if skew:
        grads = np.sqrt(grads)

    #convert the gradients to greyscale
    if greyscale:
        grads = np.mean(grads, axis=-1)

    return grads


In [ ]:
grads = standard_backprop_grads[0]

#process the gradients
relu_grads = process_grad(grads, activation="relu")
abs_grads = process_grad(grads, activation="abs")
grey_grad = process_grad(grads, greyscale=True, skew=False)


# create subplots
fig, ax = plt.subplots(1, 3, figsize=(10, 5))

# display gradients as images
ax[0].imshow(relu_grads)
ax[0].set_title("ReLU")
ax[0].axis("off")

ax[1].imshow(abs_grads)
ax[1].set_title("Abs")
ax[1].axis("off")

ax[2].imshow(grey_grad, cmap="coolwarm")
ax[2].set_title("Greyscale")
ax[2].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#replace all in-place ReLu activation with out-of-place ones
def replace_relu(model):
    for name, child in model.named_children():
        if isinstance(child, torch.nn.ReLU):
            setattr(model, name, torch.nn.ReLU(inplace=False))
            print(f"Replcaing ReLU in layer: {name}")
        else:
            replace_relu(child)

replace_relu(model)

In [ ]:
#dict to store gradeint
gradients = {}

def relu_hook(module, grad_in, grad_out, layer_name):
    """Guided Backprop Hook"""
    modified_grad = []
    
    for g in grad_in:
        if g is not None:
            modified_grad.append(torch.clamp(g, min=0.0))
        else: 
            modified_grad.append(None)

    #save gradients 
    gradients[layer_name] = modified_grad[0].detach().cpu().numpy().squeeze()

    return tuple(modified_grad)

In [ ]:
#Hook for all layers
for name, layer in model.named_modules():

    if isinstance(layer, torch.nn.ReLU):
        layer.register_backward_hook(lambda m, gi, go, n=name: relu_hook(m, gi, go, n))
        print(f"ReLU hook registed for {name}")

**Guided backprop for target logit**

In [ ]:
#reset gradient 
img_tensor = orig_img_tn.clone()
img_tensor.requires_grad_()
model.zero_grad()

#models prediction
pred = model(img_tensor)

#select class with highest score
target_class = pred.argmax()

#compute gradient w.r.t. to logit y performing backward pass
pred[:, target_class].backward()

In [ ]:
grads = img_tensor.grad.detach().cpu().numpy().squeeze()

#process the gradients
grads = process_grad(grads, activation="relu")


# create subplots
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# display gradients as images
ax[0].imshow(relu_grads)
ax[0].set_title("Standard Backprop")
ax[0].axis("off")

ax[1].imshow(grads)
ax[1].set_title("Guided_packprop")
ax[1].axis("off")


plt.tight_layout()
plt.show()

**Guided Backprop from intermediate layers**

In [ ]:
#radients from the first layer
layer = 'features.1'

#get gradeints for all feature map in layer
layer_grads = gradients[layer]
print(layer_grads.shape)

#select a random feature map
i = np.random.randint(0, layer_grads.shape[0])
feature_map_grads = layer_grads[i]

#processing the gradients 
feature_map_grads = process_grad(feature_map_grads)

#display the gradient
plt.imshow(feature_map_grads, cmap='coolwarm')
plt.axis("off")

In [ ]:
fig, ax = plt.subplots(4,5, figsize=(15,15))

for i, layer in enumerate(['features.1', 'features.6', 'features.13', 'features.22']):
    layer_grads = gradients[layer]
    print(f"{layer}: {layer_grads.shape}")

    for j in range(5):
        n_features = layer_grads.shape[0]
        r = np.random.randint(0, n_features)

        feature_map_grads = layer_grads[r]
        feature_map_grads = process_grad(feature_map_grads)

        ax[i,j].imshow(feature_map_grads, cmap="coolwarm")
        ax[i,j].set_title(f"{r} of {n_features}")
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])

    ax[i,0].set_ylabel(f"{layer}", fontsize = 15)


plt.tight_layout()